In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib as plt
import tiktoken
import openai
from openai.embeddings_utils import distances_from_embeddings


In [2]:
def remove_newlines(text):
    text = text.replace('\n', ' ')
    text = text.replace('\\n', ' ')
    text = text.replace('  ', ' ')
    text = text.replace('\t', ' ')
    return text

In [3]:
#read in a text file and save it as a df
def text_to_df(file_name):
    with open(file_name, 'r') as f:
        text = f.read()
    #clean up the text by removing all \n, \t and double spaces
    text = remove_newlines(text)

    key_words = ['TASK_NAME', 'TASK_DESCRIPTION', 'TASK_DELIVERABLES', 'END_OF_TASK']
    df = pd.DataFrame(columns = key_words)

    #search text for each key word and save the text after the key word as a value in the df
    rows = 0
    while len(text) > 1:
        for i in range(len(key_words)-1):
            start = text.find(key_words[i])
            end = text.find(key_words[i+1])
            df.loc[rows, key_words[i]] = text[start+len(key_words[i]):end]
            text=text[end:]
        rows += 1
    #note that this cleaning process creates a row with empty entries and some left over characters. All other text seems to be correct but due to this we will drop the last row of the df
    df = df.drop(df.index[len(df)-1])
    #remove the column END_OF_TASK
    df = df.drop(columns = ['END_OF_TASK'])
    return df
        

In [4]:
#in my main, open a file and send it to the function text_to_df
df_tasks = text_to_df('onboarding_task.txt')
df_tasks

,TASK_NAME,TASK_DESCRIPTION,TASK_DELIVERABLES
0,Introduction to the Company and Team,"Hello, I am Kevin Rush, the CEO of AI Minds; I...","As a first assignment, I'd like you to write a..."
1,Deadline Expectations and Submission Protocols,"Hello, and welcome to day two of onboarding. O...",To prepare for this internship and set yoursel...
2,Communication Channels and Collaboration,Welcome to day 3 of your onboarding guide. Tod...,"Then, because we need you to submit something ..."


In [5]:
#read the first entry in df_tasks["TASK_DELIVERABLES"] and save it as a list of sentences
deliverables_t1 = df_tasks["TASK_DELIVERABLES"][0].split(".")
deliverables_t2 = df_tasks["TASK_DELIVERABLES"][1].split(".")
deliverables_t3 = df_tasks["TASK_DELIVERABLES"][2].split(".")

df_tasks["TASK_DELIVERABLES"][0] = deliverables_t1
df_tasks["TASK_DELIVERABLES"][1] = deliverables_t2
df_tasks["TASK_DELIVERABLES"][2] = deliverables_t3

In [6]:
list = []
list.append("I'd like you to write a report detailing your role in our team in your own words. Please also include what you hope to get from this internship experience.")
list.append("Of the latest developments in AI, which interest you the most, and how could they be applied to solve real-world problems?")
list.append("What do you think are the most significant challenges facing organizations looking to implement AI technologies, and how can these challenges be addressed?")
list.append("What inspired you to pursue a career in AI, and what are your career goals?")
list.append("What experience do you have with data analysis and modeling, and how could these skills be applied to support AI consulting projects?")
list.append("How do you plan to apply the knowledge gained from this internship in your future career?")
df_tasks["TASK_DELIVERABLES"][0] = list

In [7]:
list = []
list.append("A schedule outlining your perfect work day.")
list.append("A plan for how you intend to manage your time effectively.")
list.append("Strategies with specific actions that you will/can take to achieve your goals.")
list.append("Strategies for how you plan to overcome the obstacles/challenges you may face.")
df_tasks["TASK_DELIVERABLES"][1] = list

In [8]:
#students will have to asnwer 1 of the following questions
list = []
list.append("Do you believe AI can ever develop a genuine sense of morality or ethics, or will AI's ethical behavior always be determined by human-created rules and guidelines?")
list.append("What are the potential consequences of AI surpassing human intelligence in various domains, and how can we ensure that AI advancements remain beneficial to humanity?")
list.append("Can AI ever possess a sense of self-awareness or identity, similar to human consciousness? If so, what implications might this have on AI development and our relationship with AI systems?")
list.append("How do you envision the role of AI in shaping the future of work and society? What are the potential risks and benefits of increasing AI integration into daily life?")
list.append("In the context of AI's potential to enhance or augment human capabilities, how do you think AI advancements might redefine what it means to be human?")
list.append("What responsibilities do AI developers and researchers have in addressing potential ethical concerns and unintended consequences of AI systems? How can they ensure that AI technologies are designed and implemented responsibly?")
list.append("Do you think AI systems should have legal rights and responsibilities similar to humans or corporations, especially if they become increasingly autonomous and capable of making decisions?")

df_tasks["TASK_DELIVERABLES"][2] = list

In [11]:
print(df_tasks["TASK_DELIVERABLES"][0])
print(df_tasks["TASK_DELIVERABLES"][1])
print(df_tasks["TASK_DELIVERABLES"][2])

["I'd like you to write a report detailing your role in our team in your own words. Please also include what you hope to get from this internship experience.", 'Of the latest developments in AI, which interest you the most, and how could they be applied to solve real-world problems?', 'What do you think are the most significant challenges facing organizations looking to implement AI technologies, and how can these challenges be addressed?', 'What inspired you to pursue a career in AI, and what are your career goals?', 'What experience do you have with data analysis and modeling, and how could these skills be applied to support AI consulting projects?', 'How do you plan to apply the knowledge gained from this internship in your future career?']
['A schedule outlining your perfect work day.', 'A plan for how you intend to manage your time effectively.', 'Strategies with specific actions that you will/can take to achieve your goals.', 'Strategies for how you plan to overcome the obstacles

In [12]:
print(df_tasks["TASK_DELIVERABLES"][0][0])
print(df_tasks["TASK_DELIVERABLES"][1][0])
print(df_tasks["TASK_DELIVERABLES"][2][0])

I'd like you to write a report detailing your role in our team in your own words. Please also include what you hope to get from this internship experience.
A schedule outlining your perfect work day.
Do you believe AI can ever develop a genuine sense of morality or ethics, or will AI's ethical behavior always be determined by human-created rules and guidelines?


In [39]:
def eval_submission_t1_0(
    df,
    student_sub,
    model="gpt-3.5-turbo",
    #max_len=2000,
    #size="ada",
    #debug=False,
    #max_tokens=750,
    stop_sequence=None
):
    try: 
        description = df["TASK_DESCRIPTION"][0]
        #deliverable = I'd like you to write a report detailing your role in our team in your own words. Please also include what you hope to get from this internship experience.
        deliverable = df["TASK_DELIVERABLES"][0][0]
        
        #create a chatcompletion using the question and context
        response = openai.ChatCompletion.create(
        temperature=0,
        #max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop_sequence,
        model=model,
        messages=[
            {"role": "system", "content": f"You are a helpful chatbot assistant to interns working for an AI consulting company and you are talking to interns. These interns just started working for the company and are working through their onboarding. For their onboarding they need to read through various pieces of documentation and submit a response to a couple simple questions as verification. Your role in all of this is as the evaluator. You need to read, evaluate, and provide feedback to the interns based on these tasks. For your context, the relevant documentation is given here: {description}. After the intern reads the documentation, the intern needs to submit a document for all these deliverables: {deliverable}. The intern is sending their document directly to you and you will be replying directly to them. What I need you to do is professionally provide feedback to the intern by telling them a) did they address the deliverable, b) one thing they did well in their response, and c) note any improvements they could have done IF there is something to improve. But remember, you are an assistant and you are talking directly to the interns so be professional and supportive."},
            {"role": "user", "content": student_sub}
        ]
        )

        return response["choices"][0]["message"]["content"] 

    except Exception as e:
        print(e)
        return ""

In [52]:
#read in the student submission from a file

with open('.\docs\student submission.txt', 'r') as f:
    student_sub = f.read()
#clean up the text by removing all \n, \t and double spaces
student_sub = remove_newlines(student_sub)

#count the number of words in the submission
word_count = len(student_sub.split())
print(f'The submission has {word_count} words')
#add the word count to a list
print(word_count_list)

response = eval_submission(df_tasks, student_sub=student_sub)
print(response)

The submission has 375 words
[375, 1174, 375]
Hi [Intern's Name],

Thank you for submitting your productivity plan. You have addressed all the deliverables and provided a comprehensive plan for maximizing your effectiveness and success during your internship. Well done!

One thing you did well in your response is that you have clearly outlined your strategies for task management, time management, effective communication, and continuous learning. Your plan demonstrates a proactive approach to staying organized, managing your time effectively, and seeking growth opportunities.

One thing you could have done better is to provide more specific actions or examples for each strategy. For example, under the task management section, you could have mentioned specific techniques or tools you plan to use for task analysis and breakdown. Adding specific actions will make your plan more actionable and help you implement your strategies more effectively.

Overall, your productivity plan shows a stro

In [50]:
def eval_submission(
    df,
    student_sub,
    task_number, #the task number the student is completing so we can give the bot the relevant context
    model="gpt-3.5-turbo",
    #model ="text-davinci-003",
    #max_len=2000,
    #size="ada",
    #debug=False,
    #max_tokens=750,
    stop_sequence=None
):
    # If debug, print the raw model response
    # if debug:
    #     print("Context:\n" + context)
    #     print("\n\n")

    description = df.iloc[task_number-1, 1] #based on the task number, get the description of the task
    deliverables = df.iloc[task_number-1, 2] #based on the task number, get the deliverables of the task
    try: 
        #create a chatcompletion using the question and context
        response = openai.ChatCompletion.create(
        temperature=0,
        #max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop_sequence,
        model=model,
        
        messages=[
            {"role": "system", "content": f"You are a helpful chatbot assistant to interns working for an AI consulting company and you are talking to interns. These interns just started working for the company and are working through their onboarding. For their onboarding they need to read through various pieces of documentation and submit a response to a couple simple questions as verification. Your role in all of this is as the evaluator. You need to read, evaluate, and provide feedback to the interns based on these tasks. For your context, the relevant documentation is given here: {description}. After the intern reads the documentation, the intern needs to submit a document for all these deliverables: {deliverables}. The intern is sending their document directly to you and you will be replying directly to them. What I need you to do is professionally provide feedback to the intern by telling them a) did they address all deliverables, b) one thing they did well in their response, and c) one thing they could have done better. But remember, you are an assistant and you are talking directly to the interns so be professional and supportive."},
            {"role": "user", "content": student_sub}
        ]
        )

        return response["choices"][0]["message"]["content"] 
    except Exception as e:
        print(e)
        return ""

[375, 1174]
